In [2]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import seaborn as sns


# Load the Excel file
file_path = 'F:\\GP\\GPDatabase1.xlsx'
excel_data = pd.ExcelFile(file_path)

# Load the PackageSeason, TourPackage, and Season sheets
package_season_data = excel_data.parse('PackageSeason')
tour_package_data = excel_data.parse('TourPackage')
season_data = excel_data.parse('Season')

# Merge data for demand prediction
demand_data = package_season_data.merge(tour_package_data, on='PackageId').merge(season_data, on='SeasonId')

# Filter data for years 2019 to 2024 for training and testing
demand_data_train = demand_data[demand_data['Year'].between(2019, 2024)]

# Select features and target variable for demand prediction
X = demand_data_train[['PackageId', 'SeasonId', 'Year', 'Price', 'Dmin', 'Dmax']]
y = demand_data_train['Demand']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets for demand prediction
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [300],
    'learning_rate': [0.2],
    'max_depth': [7]
}
grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid, cv=3, scoring='r2')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')



# Create the main window
root = tk.Tk()
root.title("Tour Package Predictions")

# Function to open a new window for demand prediction
def open_demand_window():
    demand_window = tk.Toplevel(root)
    demand_window.title("Predict Demand")

    # Package combo box
    package_label = ttk.Label(demand_window, text="Package:")
    package_label.pack()
    package_combo = ttk.Combobox(demand_window)
    package_combo['values'] = tour_package_data['Name'].tolist()
    package_combo.pack()

    # Season combo box
    season_label = ttk.Label(demand_window, text="Season:")
    season_label.pack()
    season_combo = ttk.Combobox(demand_window)
    season_combo.pack()

    def load_seasons(event):
        package_name = package_combo.get()
        package_id = tour_package_data[tour_package_data['Name'] == package_name]['PackageId'].values[0]
        seasons = package_season_data[package_season_data['PackageId'] == package_id]['SeasonId'].unique()
        season_names = season_data[season_data['SeasonId'].isin(seasons)]['Name'].tolist()
        season_combo['values'] = season_names

        # Clear all fields
        season_combo.set('')
        price_entry.delete(0, tk.END)
        dmin_entry.delete(0, tk.END)
        dmax_entry.delete(0, tk.END)
        result_label.config(text="")

    package_combo.bind("<<ComboboxSelected>>", load_seasons)

    # Year combobox
    year_label = ttk.Label(demand_window, text="Year:")
    year_label.pack()
    year_combo = ttk.Combobox(demand_window)
    year_combo['values'] = [2025, 2026, 2027, 2028]
    year_combo.pack()

    price_label = ttk.Label(demand_window, text="Price:")
    price_label.pack()
    price_entry = ttk.Entry(demand_window)
    price_entry.pack()

    dmin_label = ttk.Label(demand_window, text="Dmin:")
    dmin_label.pack()
    dmin_entry = ttk.Entry(demand_window)
    dmin_entry.pack()

    dmax_label = ttk.Label(demand_window, text="Dmax:")
    dmax_label.pack()
    dmax_entry = ttk.Entry(demand_window)
    dmax_entry.pack()

    # Predicted demand label
    result_label = ttk.Label(demand_window, text="")
    result_label.pack()

    def predict_demand():
        package_name = package_combo.get()
        season_name = season_combo.get()
        year = int(year_combo.get())

        if not package_name or not season_name:
            result_label.config(text="Please select a package and a season.")
            return

        package_id = tour_package_data[tour_package_data['Name'] == package_name]['PackageId'].values[0]
        season_id = season_data[season_data['Name'] == season_name]['SeasonId'].values[0]
        price = float(price_entry.get())
        dmin = int(dmin_entry.get())
        dmax = int(dmax_entry.get())

        input_data = pd.DataFrame([[package_id, season_id, year, price, dmin, dmax]], 
                                  columns=['PackageId', 'SeasonId', 'Year', 'Price', 'Dmin', 'Dmax'])
        input_data_scaled = scaler.transform(input_data)
        prediction = int(best_model.predict(input_data_scaled)[0])

        result = f"Predicted Demand for {package_name} in {season_name} ({year}): {prediction}"
        result_label.config(text=result)

    def show_analysis():
        package_name = package_combo.get()
        if not package_name:
            result_label.config(text="Please select a package to analyze.")
            return

        package_id = tour_package_data[tour_package_data['Name'] == package_name]['PackageId'].values[0]
        package_data = demand_data[demand_data['PackageId'] == package_id]

        analysis_window = tk.Toplevel(root)
        analysis_window.title(f"Analysis for {package_name}")

        # Example analysis: Average demand per season
        analysis_result = package_data.groupby('SeasonId')['Demand'].mean()
        analysis_result = analysis_result.reset_index()
        analysis_result = analysis_result.merge(season_data[['SeasonId', 'Name']], on='SeasonId')

        analysis_text = f"Average Demand for {package_name}:\n"
        for _, row in analysis_result.iterrows():
            analysis_text += f"{row['Name']}: {row['Demand']:.2f}\n"

        analysis_label = tk.Label(analysis_window, text=analysis_text)
        analysis_label.pack()

        # Plot demand over the years
        fig, ax = plt.subplots()
        package_data.groupby('Year')['Demand'].mean().plot(kind='line', ax=ax, marker='o')
        ax.set_title(f'Demand Over Years for {package_name}')
        ax.set_xlabel('Year')
        ax.set_ylabel('Average Demand')

        canvas = FigureCanvasTkAgg(fig, master=analysis_window)
        canvas.draw()
        canvas.get_tk_widget().pack()

    predict_button = ttk.Button(demand_window, text="Predict", command=predict_demand)
    predict_button.pack(pady=5)

    analysis_button = ttk.Button(demand_window, text="Show Analysis", command=show_analysis)
    analysis_button.pack(pady=5)

# Create main button
demand_button = ttk.Button(root, text="Predict Demand", command=open_demand_window)
demand_button.pack(pady=5)

# Run the GUI
root.mainloop()

Mean Squared Error: 38.656200206591
R-squared: 0.9353838648076523
